#### Description 

In [6]:
print('Name:Leo pauly')
print('Description: Program for implementing PINN using Darcys eq')

Name:Leo pauly
Description: Program for implementing PINN using Darcys eq


#### Imports

In [7]:
import numpy as np
import matplotlib.pyplot as plt 
import sciann as sn 
import pandas as pd
from numpy import pi
from sciann.utils.math import diff, sign, sin
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#### Darcy-PINN 


In [8]:
x = sn.Variable('x',dtype='float64')
y = sn.Variable('y',dtype='float64')
v = sn.Functional('v', [x,y], 8*[20], 'relu')
K = sn.Functional('K', [x,y], 8*[20], 'relu')
h = sn.Functional('h', [x,y], 8*[20], 'relu')

In [10]:
d1 = sn.Data(v)
d3 = sn.Data(h)
C1 = -v-K*(diff(h, x)+diff(h, y))

In [12]:
m = sn.SciModel([x,y], [d1,d3,C1])

#### Dataset

In [16]:
v_data_df=pd.read_excel('Darcy_Flux.xlsx')  
h_data_df=pd.read_excel('Hydraulic_Head.xlsx')  
K_data_df=pd.read_excel('hyd_conductivity.xlsx')  

In [17]:
x1_data=np.array(v_data_df['X'].values)
x2_data=np.array(v_data_df['Y'].values)
x1_data_K=np.array(K_data_df['X'].values)
x2_data_K=np.array(K_data_df['Y'].values)

v_data=np.array(v_data_df['VINIT'].values)
print('v_data:',v_data.shape)

K_test=np.array(K_data_df['TRANS'].values)
print('K_test:',K_test.shape)

h_data=np.array(h_data_df['FINIT'].values)
print('h_data:',h_data.shape)

v_data: (910,)
K_test: (1728,)
h_data: (910,)


In [18]:
x_data=[]
for i in range(len(x1_data)):
    x_data.append([x1_data[i]*x2_data[i]])
    
x_data=np.array(x_data)   
print('x_data shape:',x_data.shape)

x_test_K=[]
for i in range(len(x1_data_K)):
    x_test_K.append([x1_data_K[i]*x2_data_K[i]])
    
x_test_K=np.array(x_test_K)   
print('x_k data shape:',x_test_K.shape)

x_data shape: (910, 1)
x_k data shape: (1728, 1)


In [23]:
x1_train, x1_test,x2_train, x2_test, v_train, v_test, h_train, h_test = train_test_split(x1_data,x2_data, v_data, h_data, test_size=.3, random_state=1)
print('Training size (x1,x2,v,h):',x1_train.shape,x2_train.shape,v_train.shape,h_train.shape)
C1_train = 'zeros'

Training size (x1,x2,v,h): (637,) (637,) (637,) (637,)


In [26]:
v_pred = v.eval(m, [x1_test,x2_test])
h_pred = h.eval(m, [x1_test,x2_test])
K_pred = K.eval(m, [x1_data_K,x2_data_K])
print('MSE for v before training:',round(mean_squared_error(v_test,v_pred),10))
print('MSE for h before training:',round(mean_squared_error(h_test,h_pred),10))
print('MSE for K before training:',round(mean_squared_error(K_test,K_pred),10))

MSE for v before training: 0.1354483698
MSE for h before training: 0.3516024985
MSE for K before training: 9.42581e-05


#### Training

In [30]:
learning_rate=0.001
epochs=200
history = m.train([x1_train,x2_train], [v_train,h_train,C1_train], learning_rate=learning_rate, epochs=epochs, verbose=1,batch_size=10)


Total samples: 637 
Batch size: 10 
Total batches: 64 

Epoch 1/200
64/64 [==============================] - 0s 5ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0317 - v_loss: 0.0091 - h_loss: 0.0139 - sub_2_loss: 0.0088 - lr: 0.0010 - time: 0.3571
Epoch 2/200
64/64 [==============================] - 0s 6ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0310 - v_loss: 0.0084 - h_loss: 0.0133 - sub_2_loss: 0.0093 - lr: 0.0010 - time: 0.4035
Epoch 3/200
64/64 [==============================] - 0s 6ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0276 - v_loss: 0.0072 - h_loss: 0.0130 - sub_2_loss: 0.0073 - lr: 0.0010 - time: 0.4050
Epoch 4/200
64/64 [==============================] - 0s 5ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0275 - v_loss: 0.0068 - h_loss: 0.0133 - sub_2_loss: 0.0074 - lr: 0.0010 - time: 0.3927
Epoch 5/200
64/64 [==============================] - 0s 5ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0268 - v_loss: 0.0065 - h_loss: 0.0134 - sub_2_loss: 0.0

64/64 [==============================] - 0s 5ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0183 - v_loss: 0.0028 - h_loss: 0.0109 - sub_2_loss: 0.0046 - lr: 0.0010 - time: 0.3990
Epoch 42/200
64/64 [==============================] - 0s 6ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0200 - v_loss: 0.0029 - h_loss: 0.0112 - sub_2_loss: 0.0059 - lr: 0.0010 - time: 0.4035
Epoch 43/200
64/64 [==============================] - 0s 6ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0196 - v_loss: 0.0030 - h_loss: 0.0109 - sub_2_loss: 0.0057 - lr: 0.0010 - time: 0.4026
Epoch 44/200
64/64 [==============================] - 0s 6ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0188 - v_loss: 0.0027 - h_loss: 0.0111 - sub_2_loss: 0.0051 - lr: 0.0010 - time: 0.4013
Epoch 45/200
64/64 [==============================] - 0s 6ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0190 - v_loss: 0.0027 - h_loss: 0.0109 - sub_2_loss: 0.0054 - lr: 0.0010 - time: 0.4090
Epoch 46/200
64/64 [=============

64/64 [==============================] - 0s 5ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0154 - v_loss: 0.0017 - h_loss: 0.0097 - sub_2_loss: 0.0040 - lr: 0.0010 - time: 0.3990
Epoch 83/200
64/64 [==============================] - 0s 5ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0159 - v_loss: 0.0020 - h_loss: 0.0097 - sub_2_loss: 0.0042 - lr: 0.0010 - time: 0.3979
Epoch 84/200
64/64 [==============================] - 0s 6ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0142 - v_loss: 0.0016 - h_loss: 0.0094 - sub_2_loss: 0.0031 - lr: 0.0010 - time: 0.4025
Epoch 85/200
64/64 [==============================] - 0s 6ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0147 - v_loss: 0.0013 - h_loss: 0.0098 - sub_2_loss: 0.0035 - lr: 0.0010 - time: 0.4015
Epoch 86/200
64/64 [==============================] - 0s 6ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0171 - v_loss: 0.0025 - h_loss: 0.0099 - sub_2_loss: 0.0047 - lr: 0.0010 - time: 0.4011
Epoch 87/200
64/64 [=============

57/64 [=========================>....] - ETA: 0s - batch: 28.0000 - size: 9.9474 - loss: 0.0128 - v_loss: 0.0011 - h_loss: 0.0085 - sub_2_loss: 0.0033
Epoch 00123: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
64/64 [==============================] - 0s 5ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0130 - v_loss: 0.0011 - h_loss: 0.0087 - sub_2_loss: 0.0031 - lr: 0.0010 - time: 0.3975
Epoch 124/200
64/64 [==============================] - 0s 6ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0137 - v_loss: 0.0012 - h_loss: 0.0086 - sub_2_loss: 0.0039 - lr: 5.0000e-04 - time: 0.4015
Epoch 125/200
64/64 [==============================] - 0s 5ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0127 - v_loss: 0.0011 - h_loss: 0.0085 - sub_2_loss: 0.0031 - lr: 5.0000e-04 - time: 0.3952
Epoch 126/200
64/64 [==============================] - 0s 6ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0114 - v_loss: 8.4589e-04 - h_loss: 0.0082 - sub_2_loss: 0.0023 - lr: 5.000

64/64 [==============================] - 0s 5ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0107 - v_loss: 8.0358e-04 - h_loss: 0.0073 - sub_2_loss: 0.0026 - lr: 5.0000e-04 - time: 0.3956
Epoch 163/200
64/64 [==============================] - 0s 6ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0108 - v_loss: 8.1894e-04 - h_loss: 0.0073 - sub_2_loss: 0.0027 - lr: 5.0000e-04 - time: 0.4037
Epoch 164/200
64/64 [==============================] - 0s 6ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0129 - v_loss: 7.8169e-04 - h_loss: 0.0076 - sub_2_loss: 0.0045 - lr: 5.0000e-04 - time: 0.4022
Epoch 165/200
64/64 [==============================] - 0s 5ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0123 - v_loss: 0.0014 - h_loss: 0.0073 - sub_2_loss: 0.0035 - lr: 5.0000e-04 - time: 0.3971
Epoch 166/200
64/64 [==============================] - 0s 5ms/step - batch: 31.5000 - size: 9.9531 - loss: 0.0127 - v_loss: 8.5234e-04 - h_loss: 0.0075 - sub_2_loss: 0.0043 - lr: 5.0000e-04 - time: 

#### Testing 

In [29]:
v_pred = v.eval(m, [x_test])
h_pred = h.eval(m, [x_test])
K_pred = K.eval(m, [x_test_K])
print('MSE for v after training:',round(mean_squared_error(v_test,v_pred),10))
print('MSE for h after training:',round(mean_squared_error(h_test,h_pred),10))
print('MSE for K after training:',round(mean_squared_error(K_test,K_pred),10))

NameError: name 'x_test' is not defined